In [ ]:
base_dir = '/home/vco/Datasets'
folders = ['PDC19'] #, 'PDC20', 'PDC21', 'PDC22']
# special_folders = ['PDC21', 'PDC22']
sub_folders = ['Ground Truth Logs', 'Test Logs', 'Training Logs']

for folder in folders:
    print(f'[i] Started for {folder}')
    for log_name in os.listdir(os.path.join(base_dir, folder, sub_folders[0])):
        print(f'[i] {log_name}')

In [ ]:
configs = [{
            'timed': False,
            'pending': False,
            'variant': Variants.DCR_BASIC
        },{
            'inBetweenRels': True,
            'timed': False,
            'pending': False,
            'variant': Variants.DCR_SUBPROCESS_ME,
        }
]
folder = folders[0]
log_number = 7
log_name = f'pdc_2019_{log_number}.xes'
gt = pm4py.read_xes(os.path.join(base_dir, folder, sub_folders[0], log_name), return_legacy_log_object=True)
train = pm4py.read_xes(os.path.join(base_dir, folder, sub_folders[2], log_name),
                       return_legacy_log_object=True)
train_df = pm4py.convert_to_dataframe(train)
gt_df = pm4py.convert_to_dataframe(gt)

dcrs = benchmark_event_log_from_configs(train,dcr_title=f"debug_on_pdc19_{log_number}",result_file_prefix=f"debug_on_pdc19_{log_number}",configs=configs)

In [ ]:
from pm4py.algo.discovery.dcr_discover.variants.discover_subprocess_mutual_exclusion import get_subprocess_log

sp_log = get_subprocess_log(train,dcrs[1]['subprocesses'])
sp_log_df = pm4py.convert_to_dataframe(sp_log)

In [ ]:
len(train_df)==len(sp_log_df)

In [ ]:
trace_no = 1
train_events = train_df[train_df['case:concept:name']==str(trace_no)]['concept:name'].tolist()
sp_events = sp_log_df[sp_log_df['case:concept:name']==str(trace_no)]['concept:name'].tolist()

for e in train_events:
    print(e,end=' ')
print()
for e in sp_events:
    print(e,end=' ')
print()
for n,es in dcrs[1]['subprocesses'].items():
    print(f'{n}: {es}')

In [ ]:
eval_dcr_on_pdc.compare_two_models(dcrs[0],dcrs[1],gt)

In [17]:
trace_no = 13
isPos = gt_df[gt_df['case:concept:name']==str(trace_no)]['case:pdc:isPos'].tolist()[0]
events = gt_df[gt_df['case:concept:name']==str(trace_no)]['concept:name'].to_list()
sem_test_1 = semantics_obj.DcrSemantics(dcrs[0],cmd_print=True)
sem_test_2 = semantics_obj.DcrSemantics(dcrs[1],cmd_print=True)

av s j r d d aj m b i ac ah m ab ab as 

In [22]:
print(sorted(sem_test_1.enabled()))
print(sorted(sem_test_2.enabled_atomic_events()))

['aj', 'al', 'am', 'ap', 'at', 'av', 'b', 'c', 'd', 'f', 'j', 'm', 'n', 'o', 'q', 'r', 'u', 'x', 'y', 'z']
['a', 'aj', 'al', 'am', 'ap', 'av', 'b', 'c', 'd', 'm', 'n', 'o', 'q', 'r', 'u', 'y']


In [21]:
to_execute = 's'
sem_test_1.execute(to_execute)
sem_test_2.execute(to_execute)

(True, datetime.timedelta(0))

In [ ]:
for event in events:
    ex1,_= sem_test_1.execute(event)
    ex2,_= sem_test_2.execute(event)
    if ex1 != ex2:
        print(event)
    elif ex1 == ex2 and ex1 is False and isPos is False:
        break

# The new solution to subprocesses based on mutual exclusions

In [51]:
self_excluding = set()
for e in events:
    if rel_matrices['excludesTo'].at[e,e] == 1:
        self_excluding.add(e)

In [53]:
mutually_excluding = []
for e in self_excluding:
    for e_prime in self_excluding:
        if e!=e_prime and \
        rel_matrices['excludesTo'].at[e,e_prime] == 1 and \
        rel_matrices['excludesTo'].at[e_prime,e]== 1:
            if (e,e_prime) not in mutually_excluding and (e_prime,e) not in mutually_excluding:
                if e == 'ac' or e_prime == 'ac':
                    print((e,e_prime))
                mutually_excluding.append((e,e_prime))

In [67]:
G = nx.from_edgelist(mutually_excluding)
max_sp = len(self_excluding)
cliques = set(frozenset(s) for s in nx.enumerate_all_cliques(G) if len(s) > 1)
sps = []
while max_sp>1:
    cliques_at_len = set(frozenset(s) for s in nx.enumerate_all_cliques(G) if len(s) == max_sp)
    if len(cliques_at_len)>0:
        sp = list(cliques_at_len)[0]
        sps.append(sp)
        for e in sp:
            G.remove_node(e)
    max_sp -= 1